In [2]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.1 MB/s eta 0:00:00


In [3]:
import optuna
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# Generate a toy dataset
X, y = make_classification(
    n_samples=2000,
    n_features=20,
    n_informative=15,
    n_classes=2,
    random_state=42
)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


In [5]:
def build_model(trial, input_dim):
  model = keras.Sequential()
  model.add(layers.Input(shape=(input_dim,)))

  n_layers = trial.suggest_int("n_layers",1,4)

  units = trial.suggest_int('units', 32, 256, step=32)

  dropout = trial.suggest_float("dropout", 0.0, 0.5)

  for i in range(n_layers):
    model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dropout(dropout))

  model.add(layers.Dense(1, activation='sigmoid'))

  lr = trial.suggest_float('lr', 1e-4,1e-2, log=True)

  model.compile(
      optimizer = keras.optimizers.Adam(learning_rate=lr),
      loss = 'binary_crossentropy',
      metrics=['accuracy']
  )

  return model

In [6]:
def objective(trial):
  tf.keras.backend.clear_session()

  model = build_model(trial, input_dim=X_train.shape[1])

  batch_size = trial.suggest_categorical('batch_size', [32,64,128])

  history = model.fit(
      X_train,
      y_train,
      validation_data = (X_val, y_val),
      batch_size=batch_size,
      epochs=20
  )

  val_acc = max(history.history['val_accuracy'])
  return val_acc

In [7]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials=30)

print("Best trial")
print(" Validation Accuracy:", study.best_value)
print(" Best Hyperparameters:")
for k,v in study.best_params.items():
  print(f"{k}:{v}")

[I 2026-01-03 07:10:06,130] A new study created in memory with name: no-name-b2d671f9-2b50-464f-b514-0c9d96c0f323


Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7095 - loss: 0.5523 - val_accuracy: 0.8950 - val_loss: 0.2851
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8668 - loss: 0.3166 - val_accuracy: 0.9200 - val_loss: 0.2043
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9140 - loss: 0.2254 - val_accuracy: 0.9300 - val_loss: 0.1443
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9247 - loss: 0.2182 - val_accuracy: 0.9475 - val_loss: 0.1325
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9343 - loss: 0.1757 - val_accuracy: 0.9625 - val_loss: 0.1221
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9488 - loss: 0.1630 - val_accuracy: 0.9625 - val_loss: 0.1030
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9563 - loss: 0.1345 - val_accuracy: 0.9650 - val_loss: 0.0964
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9503 - loss: 0.1304 - val_accuracy: 0.9625 - val

[I 2026-01-03 07:10:16,699] Trial 0 finished with value: 0.9750000238418579 and parameters: {'n_layers': 3, 'units': 256, 'dropout': 0.4680976307654186, 'lr': 0.0025443371743120147, 'batch_size': 32}. Best is trial 0 with value: 0.9750000238418579.


Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6768 - loss: 0.5813 - val_accuracy: 0.8675 - val_loss: 0.3427
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8618 - loss: 0.3417 - val_accuracy: 0.9025 - val_loss: 0.2429
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9017 - loss: 0.2566 - val_accuracy: 0.9325 - val_loss: 0.1823
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9214 - loss: 0.1940 - val_accuracy: 0.9500 - val_loss: 0.1324
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9484 - loss: 0.1628 - val_accuracy: 0.9575 - val_loss: 0.1129
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9489 - loss: 0.1409 - val_accuracy: 0.9650 - val_loss: 0.0909
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9615 - loss: 0.1286 - val_accuracy: 0.9575 - val_loss: 0.0973
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9672 - loss: 0.1096 - val_accuracy: 0.9700 - val_loss:

[I 2026-01-03 07:10:24,392] Trial 1 finished with value: 0.9800000190734863 and parameters: {'n_layers': 2, 'units': 160, 'dropout': 0.24680664984628425, 'lr': 0.0013185483445156466, 'batch_size': 32}. Best is trial 1 with value: 0.9800000190734863.


...

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6175 - loss: 0.6331 - val_accuracy: 0.8075 - val_loss: 0.4415
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8234 - loss: 0.4186 - val_accuracy: 0.8775 - val_loss: 0.3163
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8765 - loss: 0.2981 - val_accuracy: 0.9000 - val_loss: 0.2439
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9257 - loss: 0.2304 - val_accuracy: 0.9075 - val_loss: 0.1919
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9360 - loss: 0.1737 - val_accuracy: 0.9375 - val_loss: 0.1583
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9385 - loss: 0.1811 - val_accuracy: 0.9525 - val_loss: 0.1262
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9564 - loss: 0.1357 - val_accuracy: 0.9625 - val_loss: 0.1134
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9633 - loss: 0.1191 - val_accuracy: 0.9675 - val_loss

[I 2026-01-03 07:13:57,413] Trial 28 finished with value: 0.9750000238418579 and parameters: {'n_layers': 3, 'units': 128, 'dropout': 0.06630454158693047, 'lr': 0.0008341776131232253, 'batch_size': 64}. Best is trial 16 with value: 0.9825000166893005.


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5585 - loss: 0.6779 - val_accuracy: 0.7700 - val_loss: 0.4985
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7903 - loss: 0.4829 - val_accuracy: 0.8425 - val_loss: 0.3474
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8449 - loss: 0.3595 - val_accuracy: 0.8775 - val_loss: 0.2933
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8701 - loss: 0.3151 - val_accuracy: 0.9050 - val_loss: 0.2436
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9069 - loss: 0.2552 - val_accuracy: 0.9150 - val_loss: 0.2045
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9279 - loss: 0.2150 - val_accuracy: 0.9425 - val_loss: 0.1651
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9269 - loss: 0.2171 - val_accuracy: 0.9275 - val_loss: 0.1489
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9325 - loss: 0.1620 - val_accuracy: 0.9500 - val_loss

[I 2026-01-03 07:14:03,985] Trial 29 finished with value: 0.9700000286102295 and parameters: {'n_layers': 4, 'units': 64, 'dropout': 0.206790949318421, 'lr': 0.0035236887208974356, 'batch_size': 128}. Best is trial 16 with value: 0.9825000166893005.


Best trial
 Validation Accuracy: 0.9825000166893005
 Best Hyperparameters:
n_layers:4
units:160
dropout:0.20706965343433087
lr:0.0017870315078298735
batch_size:32


In [8]:
best_params = study.best_params

final_model = keras.Sequential()
final_model.add(layers.Input(shape=(X_train.shape[1],)))

for i in range(best_params['n_layers']):
  final_model.add(layers.Dense(best_params['units'],activation='relu'))
  final_model.add(layers.Dropout(best_params['dropout']))

final_model.add(layers.Dense(1, activation='sigmoid'))

final_model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=best_params['lr']),
    loss = 'binary_crossentropy',
    metrics=['accuracy']
)

final_model.fit(
    X_train,
    y_train,
    batch_size=best_params['batch_size'],
    epochs=30,
    validation_data=(X_val,y_val),
    callbacks=keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
)

Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.6471 - loss: 0.5971 - val_accuracy: 0.8900 - val_loss: 0.3078
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8758 - loss: 0.3112 - val_accuracy: 0.9175 - val_loss: 0.1727
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9337 - loss: 0.1982 - val_accuracy: 0.9575 - val_loss: 0.1307
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9401 - loss: 0.1476 - val_accuracy: 0.9650 - val_loss: 0.1083
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9634 - loss: 0.1093 - val_accuracy: 0.9650 - val_loss: 0.1153
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9657 - loss: 0.1353 - val_accuracy: 0.9625 - val_loss: 0.0911
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9685 - loss: 0.0877 - val_accuracy: 0.9625 - val_loss: 0.0882
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9826 - loss: 0.0691 - val_accuracy: 0.9725 - va